In [1]:
import sys
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/')
from models.ccmodel import CCModel
from models.rsmodel import RSModel
from models.bomodel import BOModel
from models.confi import *
from models.utilfuncs import *
from graph import *
from data_analysis import *
import pandas as pd
from tqdm import tqdm
import random

# Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = "/home/mz1482/project/vt-bayesian-opt-bopt_debug/data/simu_data_4000/"
# Reading in the ECGs and labels
aucs = pd.read_csv(data_path+"Heart1/Heart1_AUCS_4000.csv", header=None).to_numpy()
ecgs = pd.read_csv(data_path+"Heart1/Heart1_SimuData_4000_200Cropped.csv", header=None).to_numpy()
labels = pd.read_csv(data_path+"Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
uvc = pd.read_csv(data_path+"UVC/Coord1_UVC_4000.csv", header=None).to_numpy()

idx=lv_rv(uvc,-1)
uvc_lv=uvc[idx]
uvc_lv = uvc_lv[:,0:3]
ecgs = ecgs[idx]
labels = labels[idx]
aucs = aucs[idx]

In [3]:
for i in rangeecgs.shape[1]

SyntaxError: invalid syntax (<ipython-input-3-53e089b8e187>, line 1)

In [ ]:
len()

In [ ]:
total_cases = 0
alle, all_points = [], []
# cc_euclids = [[] for _ in range(NUM_STEPS + 21)]                    # Random init CC arrays
cc_euclids = []
cc_successes, cc_avg_sites = [], []
cc_drop = []
# Set mm threshold for finding nearest
mm_thres = 200000

In [ ]:
def get_random_dataset(data, coords):
    """
    Handles getting the random dataset for the randomly initialized models
    :param target_coord: target coordinate
    :param data: full dataset for a patient
    :param coords: full labels for a patient
    :return: x, y of size 4
    """
    indices = np.random.choice(range(0, data.shape[0]), NUM_POINTS_START, replace=False)
    return data[indices], coords[indices]

In [ ]:
def model_run(model, x, y, train, labels, target, target_coord, target_raw, successes, avg_sites, all_euclids):
    """
    Handles running a single runthrough of a given model on a target site
    Train and labels are the initialized sets to start with (either random or by segment prediction)
    """
    # Run the training loop for the model
    euclids, preds, sites, success, nsites = model.run(x, y, train, labels, target, target_coord, target_raw)
    sites = np.asarray(sites)

    # Add to arrays if successful and add the per steps error
    if success:
        successes.append(euclids[-1])
        avg_sites.append(nsites)
    for j in range(len(euclids)):
        all_euclids[j].append(euclids[j])
        # if j != 0:
        #     drop.append(euclids[j] - euclids[j - 1])
    return successes, avg_sites, all_euclids, sites

In [ ]:
cc_model = CCModel(leads=LEADS, steps=NUM_STEPS, svr_c=SVR_C, cc=CC_THRES, cc_succ=CC_SUCC,
                   mm=mm_thres, samp_raw=ecgs, samp_coords=labels)
rs_model = RSModel(steps=NUM_STEPS, svr_c=SVR_C, samp_raw=ecgs, samp_coords=labels, cc_succ=CC_SUCC)

In [ ]:
exp = np.random.randint(0,len(labels),1)
for n in range(len(exp)):
    idx = exp[n]
    target = aucs[idx]
    target_coord = labels[idx]
    target_raw = ecgs[idx]
    # target_uvc = uvc[idx]
#     labels, aucs, ecgs=closer_points(target_coord,ecgs,labels,aucs,40)

    # Drop the target from the training set
    if idx == 0:
        x, y, raw = aucs[idx + 1:, :], labels[idx + 1:], ecgs[idx + 1:, :]
    else:
        x = np.concatenate((aucs[:idx, :], aucs[idx + 1:, :]))
        y = np.concatenate((labels[:idx], labels[idx + 1:]))
        raw = np.concatenate((ecgs[:idx], ecgs[idx + 1:]))
    random_x, random_y = get_random_dataset(x, y)

    cc_euclids, cc_preds, cc_sites, success, num_sites,no_of_lead = cc_model.run(x, y, random_x, random_y,
                                                             target, target_coord, target_raw)

In [ ]:
success

In [ ]:
no_of_lead

In [ ]:
num_sites

In [ ]:
ecgs.shape

In [ ]:
target

In [ ]:
euclid = np.array([25,42,3])

In [ ]:
inds = np.argsort(-euclid)

In [ ]:
euclid[inds]

In [ ]:
target_raw.shape

In [ ]:
target_raw = np.reshape(target_raw, [12, -1])

In [ ]:
for i in range(10):
    if i==3:
        continue
    print(i)

In [ ]:
exp = np.random.randint(0,3999,5)

In [ ]:
idx = exp[1]

In [ ]:

nn_labels, nn_aucs, nn_ecgs=closer_points(target_coord,ecgs,labels,aucs,20)

In [ ]:
nn_ecgs.shape

In [ ]:
aucs.shape